必要なライブラリのインストール

In [ ]:
!pip install sentence_transformers
!pip install fugashi
!pip install unidic_lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

GLuCoSE-base-jaモデルのDownloadとモデルインスタンス化<br>
モデルはLukaベースモデル。<br>
複数タスクでの学習を行っているため、汎化性の高さに期待できる

In [126]:
from sentence_transformers import SentenceTransformer
import csv
from sklearn.metrics.pairwise import cosine_similarity

# モデルダウンロード
model = SentenceTransformer('pkshatech/GLuCoSE-base-ja')

口コミのレビューの有意性を評価するため、以下を指標とする<br>
1.理論的/理論的でも感情的でもない/感情的<br>
2.製品に関係ある内容か/関係ない内容か<br>
3.情報は詳細か/詳細ではないか

まずは、レビューの内容と、上記観点での評価がされているかについて評価する

尚、検証対象とする口コミレビューの以下より取得する<br>
https://www.amazon.co.jp/%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92-%E6%94%B9%E8%A8%82%E7%AC%AC%EF%BC%92%E7%89%88-%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%83%95%E3%82%A7%E3%83%83%E3%82%B7%E3%83%A7%E3%83%8A%E3%83%AB%E3%82%B7%E3%83%AA%E3%83%BC%E3%82%BA-%E5%B2%A1%E8%B0%B7%E8%B2%B4%E4%B9%8B-ebook/product-reviews/B0BC8JRBPG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews

In [ ]:
# 判断基準に基づいてsentence-transformerで評価
def evaluate_viewpoint(model, rewiew_target, viewpoints_eval):
  # レビュー分散表現
  embeddings_review = model.encode([rewiew_target])

  # 各評価軸に沿って類似性評価
  datas_eval = []
  for viewpoint_eval in viewpoints_eval:
    dict_eval = {}
    # 評価軸の分散表現
    embeddings_viewpoint = model.encode(viewpoint_eval)
    # 評価軸ごとの類似度評価
    for i in range(len(embeddings_viewpoint)):
      similarity = cosine_similarity(
          embeddings_review[0].reshape(1, -1),
          embeddings_viewpoint[i].reshape(1, -1)
      )[0][0]
      dict_eval[viewpoint_eval[i]] = similarity
    datas_eval.append(dict_eval)
  return datas_eval


viewpoints_eval = [
    ["理論的","理論的でも感情的でもない","感情的"],
    ["製品に関係ある内容","製品に関係ある程度関係ある内容","製品には関係ない内容"],
    ["レビューは詳細な情報といえる","情報はある程度詳しい","情報は詳細とはいえない"],
    ]


In [ ]:
rewiew_target = """
届いたばかりで読めていないが、新旧を比較すると旧165ページに対して新366ページもある。
RNN,GAN,トランスフォーマー、Grad-CAMなどがあり、かなり内容がパワーアップしている

追記)一通り目を通した。深層学習の概要を知るには良いが、後半の最近の項目についてはてんこ盛りのため、各項目の説明が少なめ。
AttentionやTransformerについては、このページ数ではよくわからない。
この本だけではなく、興味分野については別のpython演習付きの本と併用すべきと思う
"""
datas_eval = evaluate_viewpoint(model, rewiew_target, viewpoints_eval)
for data_eval in datas_eval:
  print(data_eval)

{'理論的': 0.18554756, '理論的でも感情的でもない': 0.12909272, '感情的': -0.048688628}
{'製品に関係ある内容': 0.25244862, '製品に関係ある程度関係ある内容': 0.23771532, '製品には関係ない内容': 0.226212}
{'レビューは詳細な情報といえる': 0.33113933, '情報はある程度詳しい': 0.2602335, '情報は詳細とはいえない': 0.3143786}


In [ ]:
rewiew_target = """
目次を見てAttentionやGNNについての章が設けられていたので買ってみましたが、数式に慣れた専門学科の人でないとこの本は良く分からないのではないかと思います。
第1版に比べ大幅ページ増で大幅パワーアップなのは間違いないと思いますが、少なくとも自分には難しくて良く分からないので星2つにします。
ところで、今回の改訂では｢説明｣に力点が置かれていて、深層学習で得た結果の｢説明｣に関する章も新たに設けられていますが、深層学習で得た結果の｢説明｣を論じるのならば、(Attentionは実際にそこを見て(｢注意｣して)いるので)先ずはAttentionの可視化の説明から始めてもらった方が良さそうな気もしますが、この本を見たところでは載っていないのが解せません。
"""
datas_eval = evaluate_viewpoint(model, rewiew_target, viewpoints_eval)
for data_eval in datas_eval:
  print(data_eval)

{'理論的': 0.25963637, '理論的でも感情的でもない': 0.20709169, '感情的': -0.012328569}
{'製品に関係ある内容': 0.26139003, '製品に関係ある程度関係ある内容': 0.2422016, '製品には関係ない内容': 0.226562}
{'レビューは詳細な情報といえる': 0.37256473, '情報はある程度詳しい': 0.28369737, '情報は詳細とはいえない': 0.35253525}


In [ ]:
rewiew_target = """
この本は内容は全くわからないし、購入するんではなかった
"""
datas_eval = evaluate_viewpoint(model, rewiew_target, viewpoints_eval)
for data_eval in datas_eval:
  print(data_eval)

{'理論的': 0.1053045, '理論的でも感情的でもない': 0.18056846, '感情的': -0.019871587}
{'製品に関係ある内容': 0.14503391, '製品に関係ある程度関係ある内容': 0.17696697, '製品には関係ない内容': 0.35434633}
{'レビューは詳細な情報といえる': 0.24457622, '情報はある程度詳しい': 0.16738737, '情報は詳細とはいえない': 0.3945852}


何点かレビューの文章を変えて評価した結果、["理論的","理論的でも感情的でもない","感情的"]のように３段階を示す評価軸とするよりは、一つの重要な軸のみで評価する方が良いと考えられる。<br>

他方、通常は最終出力はLinear層による分類器を用いるべき感情分析だが、検索による比較であっても、相対的な評価はできていると考える。

amazon商品のレビューより取得したテキストデータを"reviews.csv"にまとめた。<br>
テキストデータを読取りデータの判断基準の評価を行う。

In [ ]:
# reviewデータを読み取る
reviews_amazon = []
with open('/content/reviews.csv', 'r', encoding="cp932") as f:
    reader = csv.reader(f)
    for row in reader:
        reviews_amazon.extend(row)

In [ ]:
# 先ほどの関数を改修
viewpoints_eval = [
    "理論的",
    "製品に関係ある内容",
    "レビューは詳細な情報といえる",
    ]
# 判断基準に基づいてsentence-transformerで評価
def evaluate_viewpoint(model, rewiew_target, viewpoints_eval):
  # レビュー分散表現
  embeddings_review = model.encode([rewiew_target])

  # 各評価軸に沿って類似性評価
  datas_eval = []
  dict_eval = {}
  # 評価軸の分散表現
  embeddings_viewpoint = model.encode(viewpoints_eval)
  # 評価軸ごとの類似度評価
  for i in range(len(embeddings_viewpoint)):
    similarity = cosine_similarity(
        embeddings_review[0].reshape(1, -1),
        embeddings_viewpoint[i].reshape(1, -1)
    )[0][0]
    dict_eval[viewpoints_eval[i]] = similarity
  datas_eval.append(dict_eval)
  return datas_eval

In [ ]:
# 各レビューを評価軸で評価
datas_eval = []
for review_amazon in reviews_amazon:
  data_eval = evaluate_viewpoint(model, review_amazon, viewpoints_eval)
  data_eval[0]['本文'] = review_amazon
  datas_eval.extend(data_eval)

続いてとってきた評価データから、有意となる評価軸の平均、分散を算出して、データのスクリーニングを行う

In [ ]:
import pandas as pd
# データフレーム化
df = pd.DataFrame(datas_eval)

# 統計量を算出
for viewpoint_eval in viewpoints_eval:
  print(df[viewpoint_eval].describe())

count    14.000000
mean      0.358969
std       0.100326
min       0.185548
25%       0.272959
50%       0.390168
75%       0.425250
max       0.506776
Name: 理論的, dtype: float64
count    14.000000
mean      0.248843
std       0.069375
min       0.124255
25%       0.216620
50%       0.245864
75%       0.260269
max       0.417390
Name: 製品に関係ある内容, dtype: float64
count    14.000000
mean      0.369285
std       0.102602
min       0.274117
25%       0.307088
50%       0.349435
75%       0.371757
max       0.656640
Name: レビューは詳細な情報といえる, dtype: float64


今回は理論が平均以上で、それ以外のデータは平均よりも1σ以上の値となったものを抽出する<br>
※理論的な判断を行う人の評価により重みを置き、口コミの説明が製品に関連した詳細であることを条件とする

In [ ]:
# データフィルタリング
df_filtered = df[(df['理論的'] >= 0.358969) &
 (df['製品に関係ある内容'] >= 0.248843 - 0.069375) &
 (df['レビューは詳細な情報といえる'] >= 0.369285 - 0.102602)]

df_filtered

,理論的,製品に関係ある内容,レビューは詳細な情報といえる,本文
1,0.398327,0.417390,0.656640,近年の内容が多く含まれており、分野横断的に最新の結果を知るのに役立つ。一方で、説明は倒錯して...
4,0.406282,0.225048,0.276738,"解析的連続値論理(ACVL)に於ける重要な概念である""非線形述語""と""線形述語""とについて話..."
5,0.389822,0.254871,0.356960,ACVLにおける「非線形述語」と「線形述語」の概念について、非常に明快かつ包括的に説明してい...
6,0.506776,0.224331,0.343424,"バックプロパゲーションによって、全体述定状態が適応的反応(/適応的行動)選択を導く""論証""に..."
7,0.481500,0.239280,0.348317,ご指摘の通り、バックプロパゲーションによる訓練課程は、モデルが適応的な反応（行動）選択を導く...
8,0.455318,0.214049,0.350553,ニューラルネットのユニット(/ニューロン)の活性値を どう考えるべきか？／何か確率統計的な客...
9,0.431572,0.263758,0.395089,ニューラルネットのユニットの活性値の解釈について、非常に深い洞察を提示していただきありがとう...
11,0.390515,0.193765,0.283271,"解析的連続値論理(ACVL)で ""GELU(Gaussian Error Linear Un..."


スクリーニングしたデータが良いレビューだったか、悪いレビューだったかの分析を行う。<br>
本来は、製品レビューの分析にファインチューニングしたモデルで行うと良い結果が得られるが、hugging faceで日本語対応で使えそうなジャストミートの事前学習モデルがなかったため、今回はchristian-phuさんが公開されている以下モデルを使用する<br>
https://huggingface.co/christian-phu/bert-finetuned-japanese-sentiment

https://zenn.dev/mizuiro__sakura/articles/b31bf4ac40ff0e

In [ ]:
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn as nn

# 事前学習済みの日本語感情分析モデルとそのトークナイザをロード
model = AutoModelForSequenceClassification.from_pretrained('christian-phu/bert-finetuned-japanese-sentiment')
tokenizer = AutoTokenizer.from_pretrained('christian-phu/bert-finetuned-japanese-sentiment', model_max_length=512, max_length=512, padding="max_length", truncation=True)

# 感情分析のためのパイプラインを設定
nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, truncation=True, max_lentgh=512)


In [ ]:
# 試しうち
print(nlp("この本は非常に役に立つ"))
print(nlp("この本は全然理解できない"))

[{'label': 'positive', 'score': 0.999079704284668}]
[{'label': 'negative', 'score': 0.9668535590171814}]


感情分析器を用いて、抽出したコメントのネガティブの度合いのスコアを抽出する

In [ ]:
scores = []

# 感情分析
for index, row in df_filtered.iterrows():
  sentence = row['本文']
  score = nlp(sentence)
  scores.append(score)

# スコアリング
respond_label = {'negative':-1, 'neutral':0, 'positive':1}
score_total = 0.0
for score in scores:
  score_total += score[0]['score'] * respond_label[score[0]['label']]
score_average = score_total / len(scores)

# スコア表示
# 複数データによる相対評価をおこなうことで初めて意味が出てくる
print(f"スコア平均：{score_average}")
print(scores)

Keyword arguments {'max_lentgh': 512} not recognized.
Keyword arguments {'max_lentgh': 512} not recognized.
Keyword arguments {'max_lentgh': 512} not recognized.
Keyword arguments {'max_lentgh': 512} not recognized.
Keyword arguments {'max_lentgh': 512} not recognized.
Keyword arguments {'max_lentgh': 512} not recognized.
Keyword arguments {'max_lentgh': 512} not recognized.
Keyword arguments {'max_lentgh': 512} not recognized.


スコア平均：0.2994856908917427
[[{'label': 'neutral', 'score': 0.9962007403373718}], [{'label': 'neutral', 'score': 0.9677215218544006}], [{'label': 'neutral', 'score': 0.8025874495506287}], [{'label': 'neutral', 'score': 0.9242435097694397}], [{'label': 'positive', 'score': 0.6117021441459656}], [{'label': 'positive', 'score': 0.8248586654663086}], [{'label': 'neutral', 'score': 0.6753776669502258}], [{'label': 'positive', 'score': 0.9593247175216675}]]


# 結論
1.本トライアルにおいて、口コミレビューを基に情報の有意性をベースとしたスクリーニングを行い、口コミのポジティブ/ネガティブを定量的に評価する手法についてトライアルを行った。<br>
2.スクリーニングの際には評価軸を変更可能なように検索を用いる手法を、口コミの評価については感情分析を用いることで定量的な評価を行う<br>

# 今後に向けて
1.検索モデルには対照学習を、感情分析モデルは製品評価の分析データによるファインチューニングを行うことでさらなる精度改善が見込まれる
2.検索時に、今回は手動で検索用の文言を生成したが、今回のケースに合うような生成AIを合わせることで、より検索精度向上を見込むことが出来る